In [ ]:
import csv, json, requests
import streamlit as st
import pandas as pd
import seaborn as sns
import data_config
import re
import numpy as np
from itertools import chain
from collections import defaultdict

In [ ]:

consumer_key = data_config.consumer_key
partition_id = data_config.partition_id

In [ ]:
def get_following(consumer_key,partition_id):
    url = data_config.get_following+partition_id
    headers = {"Authorization":"Bearer "+consumer_key}
    response = requests.get(url, headers=headers)
    data = json.loads(response.content)
    
    return data["response"]

In [ ]:

def get_permissions(consumer_key,partition_id):
    url = data_config.get_permssions+partition_id
    headers = {"Authorization":"Bearer "+consumer_key}
    response = requests.get(url, headers=headers)
    #print(response)
    data = json.loads(response.content)
    data = data["response"]["permissions"]
    return data




In [ ]:
#timeline and type of interaction 
def get_message_data(consumer_key,partition_id):
    url = data_config.get_activity+partition_id
    headers = {"Authorization":"Bearer "+consumer_key}
    response = requests.get(url, headers=headers)
    data = json.loads(response.content)
    #st.write(data)
    timeline = data["detailed_log"]
    summary = data["summary"]
    return timeline, summary

In [ ]:
#what is positions who can edit, probably not needed 
def get_cloud_personas(consumer_key,partition_id):
    url = data_config.get_personas+partition_id
    headers = {"Authorization":"Bearer "+consumer_key}
    response = requests.get(url, headers=headers)
    data = json.loads(response.content)
    return data["response"]["personas"]

In [ ]:
#remove duplicates from the list
def remove_duplicates(list, unique_list):
    for item in list:
        if item not in unique_list:
            unique_list.append(item)
    return unique_list            

In [ ]:
def remove_bracket(list):
    list1=[]
    for item in list:
        item2 = item[1:]
        list1.append(item2)
    return list1    

In [ ]:
data=get_message_data(consumer_key,partition_id)


In [ ]:
data1=list(data)
keys=data[0].keys()


In [ ]:
dates=[]
for i in keys:
    dates.append(i)

In [ ]:
persona_id=[]
for i in range(0, len(dates)):
    id=dates[i][-5:]
    persona_id.append(id)

In [ ]:
values1=[]
values=data[0].values()
for i in values:
    values1.append(i)
    

In [ ]:
#split the reponse based on the channel 
channel=[]
interaction=[]
RFI=[]
text=[]
id1=[]
id2=[]

for i in range(0, len(values1)):
    x = values1[i].split("]")
    channel.append(x[0])
    interaction.append(x[1])
    RFI.append(x[2])
    text.append(x[3])
    id1.append(x[4])
    id2.append(x[5])
    
    
    

        




In [ ]:
interaction=remove_bracket(interaction)
channel=remove_bracket(channel)
RFI=remove_bracket(RFI)
text=remove_bracket(text)
id1=remove_bracket(id1)
id2=remove_bracket(id2)

In [ ]:
#editing the text cell in more detail 
#need to remove the <br / br> 
#also need to remove the //
text2=[]
for item in text:
    edited_text=re.sub("(<br\s*/>)","", item)
    edited_text=re.sub("(<b)","", edited_text)
    edited_text=edited_text.replace("\\","")
    text2.append(edited_text)


In [ ]:
#editing the date:
dates
dates2=[]
for item in dates:
    item1=item[:-6]
    dates2.append(item1)
    

In [ ]:
df=pd.DataFrame({"date": dates2, "persona_id":persona_id,"channel": channel, "interaction": interaction, 
                 "Text":text2, "From": id1, "To":id2})

df.head()


In [ ]:
persona=get_cloud_personas(consumer_key,partition_id)

In [ ]:
persona_id
unique_persona_id=[]
for item in persona_id:
    if item not in unique_persona_id:
        unique_persona_id.append(item)

In [ ]:
#linking the ids with the unique persona, need to understand the json document that need sto do thisu

persona_keys=persona.keys()

In [ ]:
name=[]
imageURL=[]
type=[]

for item in unique_persona_id:
    name.append(persona[item]["details"]["name"])
    imageURL.append(persona[item]["details"]["imageUrl"])
    type.append(persona[item]["details"]["type"])
    
                
    

In [ ]:
persona_df=pd.DataFrame({"persona_id":unique_persona_id, "name": name, "image_url":imageURL, "type":type})

In [ ]:
persona_df

In [ ]:
#removing blank spaces
nospace_id2=list(filter(None, id2))

edited_list2 = [y for x in nospace_id2 for y in x.split(',')]


In [ ]:
#getting a list of unique_ids
all_ids=id1+edited_list2
unique_ids=[]
remove_duplicates(all_ids, unique_ids)
unique_ids

In [ ]:
twitter=[]
twitter_number=[]
twitter_followers=[]
twitter_following=[]
email=[]

gosocial_name=[]
gosocial_number=[]
gosocial_followers=[]


for item in unique_ids:
    #twitter data
    twitter.append(persona[item]["microblog"]["displayName"])
    twitter_number.append(persona[item]["microblog"]["numMessages"])
    twitter_followers.append(persona[item]["microblog"]["followers"])
    twitter_following.append(persona[item]["microblog"]["following"])
    
    #email
    email.append(persona[item]["mail"]["email"])
    
    #GoSocial
    gosocial_name.append(persona[item]["gosocial"]["displayName"])
    gosocial_number.append(persona[item]["gosocial"]["numMessages"])
    gosocial_followers.append(persona[item]["gosocial"]["followers"])

    

In [ ]:
#Player_activity_dataset

channel_df=pd.DataFrame({"From":unique_ids, "twitter name":twitter, "twitter messages":twitter_number,
             "twitter followers":twitter_followers, "twitter_following":twitter_following,
             "email":email, "goSocial name": gosocial_name, "goSocial messages": gosocial_number, "goSocial Followers":gosocial_followers
    
})

In [ ]:

channel_df.head(20)


##to_merged=channel_df[["To", ]]

In [ ]:
channel2_df=pd.DataFrame({"To":unique_ids, "From":unique_ids, "twitter name":twitter, "twitter messages":twitter_number,
             "twitter followers":twitter_followers, "twitter_following":twitter_following,
             "email":email, "goSocial name": gosocial_name, "goSocial messages": gosocial_number, "goSocial Followers":gosocial_followers
    
})

channel2_df['twitter name'] = channel2_df['twitter name'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)


In [ ]:
from_merged=channel2_df[["From", "twitter name", "email", "goSocial name"]]
from_merged.head(20)
#string replacements in the from dataset 


In [ ]:
to_merged=channel2_df[["To", "twitter name"]]
All=["All"]
All=pd.DataFrame({"To":All, "twitter name":All})
to_merged=to_merged.append(All)


In [ ]:
to_merged.head(50)
#remove emojis from the dataframe 

In [ ]:
#explode the id2 dataframe count:
to_merged["twitter name"].unique()

In [ ]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['To'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
df2 = pd.DataFrame({'date': np.repeat(df['date'], lens),
                    'persona_id': np.repeat(df['persona_id'], lens),
                    'channel': np.repeat(df['channel'], lens),
                    'interaction': np.repeat(df['interaction'], lens),
                    'Text': np.repeat(df['Text'], lens),
                    'From': np.repeat(df['From'], lens),
                    'To': chainer(df['To'])})

In [ ]:
#data - getting across it and unferstanding it
df2["Frequency"]=1

In [ ]:
df2.head(15)

In [ ]:
merged=pd.merge(df2, from_merged, on='From')
merged.head(10)

In [ ]:
merged=merged.rename(columns={"twitter name":"from"})
merged=merged.drop(columns={"From"})


In [ ]:
#fill null values in NA
merged['To'].replace('','Everyone', inplace=True)
merged['email'].replace('','None', inplace=True)
merged['goSocial name'].replace('','None', inplace=True)

In [ ]:
merged_2=pd.merge(merged,to_merged, on='To')

In [ ]:
merged_2.head(5)

In [ ]:
merged_2=merged_2.rename(columns={"twitter name":"to"})

In [ ]:
#merge persona id: 

#merging people
merged_final=pd.merge(merged_2,persona_df, on='persona_id')

In [ ]:
merged_final

In [ ]:
merged_final=merged_final[["date","name","channel", "interaction", "Text", "from", "to", "image_url", "type", "Frequency","email"]]



In [ ]:
merged_final

In [ ]:
#total interactions of each (name) and channel 
total_interactions = merged_final.groupby(['name','channel','interaction'])
total_interactions_final=pd.DataFrame(total_interactions["Frequency"].count())
total_interactions_final

In [ ]:
#personas each person had as each person (from interactions) 
total_to = merged_final.groupby(['name','to'])
total_to=pd.DataFrame(total_to["Frequency"].count())
total_to.head(50)


In [ ]:
total_from = merged_final.groupby(['name','from'])
total_from=pd.DataFrame(total_from["Frequency"].count())
total_from.head(50)

In [ ]:
total_from_channel= merged_final.groupby(['name','from','channel'])
total_from_channel=pd.DataFrame(total_from_channel["Frequency"].count())
total_from_channel.head(50)

In [ ]:
#personas most communications- channel and person 
total_group = merged_final.groupby(['name','to', "channel"])
total_to=pd.DataFrame(total_group["Frequency"].count())
total_to.head(50)

merged_final_df=merged_final


In [ ]:
#personas by the different day, interaction (to whom- as tooltip) maybe

#received communication
#receiving time and trying to get those filters to work

In [ ]:
#write out to a json file
results = defaultdict(lambda: defaultdict(dict))

for index, value in total_from.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key] 

In [ ]:
json_list=[]
newlist = list()
for i in results.keys():
    newlist.append(i)

In [ ]:
impersonations2={}
for item in newlist:
        impersonations=results[item].keys()
        values=[]
        for i in impersonations:    
            values.append(i)
            impersonations2.update({f'{item}':values})

In [ ]:
impersonations3={"impersonations": impersonations2}

In [ ]:
json_list.append(impersonations3)

In [ ]:
merged_final2=merged_final.copy()

In [ ]:
merged_final2["channel"]="All"

In [ ]:
frames=(merged_final, merged_final2)
merged3=pd.concat(frames)
merged3.head(5)

In [ ]:
total_from_channel= merged3.groupby(['name','from','channel'])
total_from_channel=pd.DataFrame(total_from_channel["Frequency"].count())


In [ ]:
results3 = defaultdict(lambda: defaultdict(dict))

for index, value in total_from_channel.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results3[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key] 

In [ ]:
results4={"from":results3}
json_list.append(results4)

In [ ]:
#grouping the datetime 
merged3.head()

In [ ]:
merged3['date'] = pd.to_datetime(merged3['date'], infer_datetime_format=True)
merged3.head()

In [ ]:
merged3_copy=merged3.copy()
merged3_copy['channel']="All"


In [ ]:
frames=[merged3, merged3_copy]
merged4=pd.concat(frames)

In [ ]:
#list of unique values and getting a JSON file for frequency of all the players 
person_list=merged4['name'].to_list()

person_list_unique=[]

for item in person_list:
    if item not in person_list_unique:
        person_list_unique.append(item)
        

In [ ]:
channel_list=merged4["channel"].to_list()
channel_list_unique=[]
for item in channel_list:
    if item not in channel_list_unique:
        channel_list_unique.append(item)
        

In [ ]:
channel_list_unique
dict2={}

In [ ]:
channel_list_unique

In [ ]:
#jsonifying the data 
for person in person_list_unique:
    frames=[]
    for channel in channel_list_unique:
        try:
            channel_1=merged4.loc[(merged4["name"]==f"{person}") &(merged4["channel"]==f'{channel}'), :]
            merged_time_all=channel_1.resample('0.3H', on='date').sum()
            merged_time_all["channel"]=f"{channel}"
            frames.append(merged_time_all)
        except:
            continue
   
    merged=pd.concat(frames)
    merged=merged.reset_index()
    merged['date']=merged['date'].astype(str)
    merged = merged.groupby(['channel', "date"])
    merged1=pd.DataFrame(merged["Frequency"].sum())
    
    results1 = defaultdict(lambda: defaultdict(dict))
    for index, value in merged1.itertuples():
        for i, key in enumerate(index):
            if i == 0:
                nested = results1[key]
            elif i == len(index) - 1:
                nested[key] = value
        else:
            nested = nested[key]
    
    
    dict2.update({f'{person}':results1})
        
    
            
            

    
    

        


In [ ]:
dict2

In [ ]:
json_list

In [ ]:
total_to_channel= merged3.groupby(['name','to','channel'])
total_to_channel=pd.DataFrame(total_to_channel["Frequency"].count())
total_to_channel

In [ ]:
results5 = defaultdict(lambda: defaultdict(dict))

for index, value in total_to_channel.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results5[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key] 

In [ ]:
results6={"to":results5}
results6

In [ ]:
json_list.append(results6)

In [ ]:
json_list.append(dict2)

In [ ]:
json_list

In [ ]:
with open('player_data.json', 'w') as fp:
    json.dump(json_list, fp)